In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from bs4 import BeautifulSoup
import json 
import selenium
PATH = "C:\Program Files (x86)\chromedriver.exe"
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import unidecode
import re
import json
import traceback

In [2]:
col_names = ["homeTeam", "awayTeam",
             "homeManager", "awayManager",
             "homeGK", "homeP1", "homeP2", "homeP3", "homeP4", "homeP5", "homeP6", "homeP7", "homeP8", "homeP9", "homeP10",
             "awayGK", "awayP1", "awayP2", "awayP3", "awayP4", "awayP5", "awayP6", "awayP7", "awayP8", "awayP9", "awayP10",
             "homeFormation", "awayFormation",
             "date", "dateId",
             "homeScore", "awayScore",
             "ORHomeGK", "ORHomeP1", "ORHomeP2", "ORHomeP3", "ORHomeP4", "ORHomeP5", "ORHomeP6", "ORHomeP7", "ORHomeP8", "ORHomeP9", "ORHomeP10",
             "ORAwayGK", "ORAwayP1", "ORAwayP2", "ORAwayP3", "ORAwayP4", "ORAwayP5", "ORawayP6", "ORAwayP7", "ORAwayP8", "ORAwayP9", "ORAwayP10"]

In [23]:
def parse_file(htmlName, week):
    
    #html = open(htmlName, "r", encoding="utf-8")
    soup = BeautifulSoup(html, "html.parser")
    data = get_data(soup)
    idPlayer = data["matchCentreData"]["playerIdNameDictionary"]
    playerId =  {v: k for k, v in idPlayer.items()}
    lineup = get_lineup(soup, idPlayer, playerId)
    formation = get_formation(soup)
    select1 = soup.find_all("div", {"id": "match-header"})
    select2 = select1[0].find("tbody")
    select3 = select2.find_all("dd")[-1].get_text()
    date = format_date(select3)
    date_id = get_date_id(format_update,format_date(select3))
    score = select2.find_all("dd")[-3].get_text().split(" : ")
    
    return {"team": get_team(soup),
            "manager": get_manager(soup),
            "lineup":lineup, 
            "formation":formation, 
            "date": date,
            "date_id":date_id, 
            "score":score}

def get_data(soup):

    script = soup.find_all("script")
    
    for i in range(30, 40):
        try:
            data = str(script[i])
            data = data.split('<script>\n        require.config.params["args"] = ')[1]
            data = data.split(';\n    </script>')[0]
            data = data.replace("matchId", '"matchId"')
            data = data.replace("matchCentreData", '"matchCentreData"')
            data = data.replace("matchCentreEventTypeJson", '"matchCentreEventTypeJson:"')
            data = data.replace("formationIdNameMappings", '"formationIdNameMappings"')
            data = json.loads(data)
            return data
        except:
            pass



def get_lineup(soup, idPlayer, playerId):
    

    
    select1 = soup.find_all("div", {"class":"pitch-field"})
    tab = []
    for index in [0, 1]:
        select2 = select1[index].find_all("div", {"class": "player"})
        ids = []
        for player in select2:
            position = player.get("style").split(";")
            x = float(position[0].split(":")[1].split("%")[0])
            y = float(position[1].split(":")[1].split("%")[0])
            ids.append([idPlayer[str(player.get("data-player-id"))], x, y])
        ids = pd.DataFrame(ids, columns=["id", "x", "y"])
        ids = ids.sort_values(by=["x", "y"])
        ids = ids.drop(["x","y"], axis=1)
        ids = ids.squeeze()
        tab.append(ids.tolist())
    
    
    return tab

def get_position(lineup, player):
    
    for i in range(len(lineup)):
        if lineup[i] is player:
            return i
        
def get_formation(soup):
    

    tab = []
    selectDiv1 = soup.find_all("div", {"id":"match-centre-header"})
    for index in [0,1]:
        tab.append(selectDiv1[0].find_all("div", {"class": "formation"})[index].get_text())
        
    return tab
        
def get_date(date):
    
    date = date.split(", ")[1]
    
    return date

update = pd.read_csv("update.csv", index_col=0).values.tolist()
format_update = []
for elem in update:
    format_update.append([elem[0], elem[1].split(" ")[1].split(",")[0], elem[1].split(" ")[0], elem[1].split(", ")[1]])
format_update.reverse()

def format_month(month):
    
    if month == "sept":
        return "Sep"
    elif month  == "oct":
        return "Oct"
    elif month == "nov":
        return "Nov"
    elif month == "déc":
        return "Dec"
    elif month == "janv":
        return "Jan"
    elif month == "févr":
        return "Feb"
    elif month == "mars":
        return "Mar"
    elif month == "avr":
        return "Apr"
    elif month == "mai":
        return "May"
    elif month == "juin":
        return "Jun"
    elif month == "juil":
        return "Jul"
    elif month == "août":
        return "Aug"
    
month_code = {
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12
}

def format_date(date):
    
    date = date.replace(".","")
    date = date.split(" ")[1]
    day = date.split("-")[0]
    month = date.split("-")[1]
    year = date.split("-")[2]
    
    return f"{format_month(month)} {day}, 20{year}"

def number_days(day, month):
    
    if month in ["Jan", "Mar", "May", "Jul", "Aug", "Oct", "Dec"]:
        return 31 - int(day)
    elif month in ["Apr", "Jun", "Sep", "Nov"]:
        return 30 - int(day)
    elif month == "Feb":
        return 28 - int(day)
    
def create_date(old, new):
    
    tab = []
    if old[2] == new[2]:
        for i in range(int(old[1]), int(new[1])):
            tab.append([old[0], i, old[2], old[3]])  
    else:
        for i in range(int(old[1]), int(old[1])+number_days(int(old[1]), old[2])):
            tab.append([old[0], i, old[2], old[3]]) 
        for i in range(1, int(new[1])):
            tab.append([old[0], i, new[2], new[3]])
        
    return tab

def get_date_id(format_update, date):
    
    tab = []
    for i in range(len(format_update)):
        try:
            tab.append(create_date(format_update[i], format_update[i+1]))
        except:
            pass
    tab = [j for i in tab for j in i]
    month = date.split(" ")[0]
    day = str(int(date.split(" ")[1].split(",")[0]))
    year = date.split(", ")[1]
    for elem in tab:

        if (str(elem[1]) == day) & (elem[2] == month) & (elem[3] == year):
           
            return elem[0]
       
    return -1

team = {
    "PSG": "73/paris-saint-germain",
    "Lyon": "66/olympique-lyonnais",
    "Monaco": "69/as-monaco",
    "Marseille": "219/olympique-de-marseille",
    "Saint-Etienne": "1819/as-saint-etienne",
    "Lille": "65/losc-lille",
    "Montpellier": "70/montpellier-hsc",
    "Nantes": "71/fc-nantes",
    "Nice": "72/ogc-nice",
    "Rennes": "74/stade-rennais-fc",
    "Bordeaux": "59/fc-girondins-de-bordeaux",
    "Strasbourg": "76/rc-strasbourg-alsace",
    "Reims": "379/stade-de-reims",
    "Angers": "1530/angers-sco",
    "Toulouse": "1809/toulouse-football-club",
    "Amiens": "1816/amiens-sc",
    "Guingamp": "62/en-avant-de-guingamp",
    "Nimes": "224/nimes-olympique",
    "Caen": "210/stade-malherbe-caen",
    "Dijon": "110569/dijon-fco"
}

season_id = {
    "21": "210054", 
    "20": "200061",
    "19": "190075",
    "18": "180084",
    "17": "170099"
    
}

def build_link_player(team, html, season_id, date_id):
    
    soup = BeautifulSoup(html, "html.parser")
    select = soup.find_all("div", {"class": "card"})
    select2 = select[0].find("tbody", {"class": "list"})
    select3 = select2.find_all("tr")
    for elem in select3:
        
        try: 
            td = elem.find_all("td")
            info = td[1].find("a").get("href").split("/")
            return f"https://sofifa.com/player/{info[2]}/{info[3]}/{season_id}?r={date_id}&set=true"
        except:
            return -1
        
            

def get_link_player(team, date_id, wk):
    link = f"https://sofifa.com/team/{team}/?r={date_id+wk}&set=true"

    driver  = webdriver.Chrome(PATH)
    driver.get(link)
    delay = 5
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, "card")))
        html = driver.page_source

    except TimeoutException:
        print("Loading took too much time")
    driver.close()
    dico = {}
    soup = BeautifulSoup(html, "html.parser")
    select = soup.find_all("tbody", {"class": "list"})[0].find_all("tr")
    for elem in select:
        value = elem.find("td", {"class": "col-name"}).find("a", {"class": "tooltip"}).get("href").split("/")
        value = f"/{value[1]}/{value[2]}/{value[3]}/{date_id}/"
        key = elem.find("td", {"class": "col-name"}).find("a", {"class": "tooltip"}).find("div").get_text()
        dico[unidecode.unidecode(key).replace("'", "")] = value

    return dico

def get_stat(link):
    
    a = link.split("=")
    b = a[1].split("&")[0]
    return_ = -1
    try:
        
        driver = webdriver.Chrome(PATH)
        driver.get(f"{a[0]}={int(b)}&set={a[2]}")
        delay = 5
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, "wrapper")))
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        select = soup.find("div", {"class": "bp3-card player"})
        select2 = select.find_all("div", {"class": "column col-3"})
        driver.close()
        check = soup.find_all("span", {"class": "bp3-button-text"})[0].get_text().split(" ")[1]
        if check == b[:2]:
            return_ = int(select2[0].get_text().split(" ")[0])
        else:
            driver = webdriver.Chrome(PATH)
            driver.get(f"{a[0]}={int(b)+1}&set={a[2]}")
            delay = 5
            myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME, "wrapper")))
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            select = soup.find("div", {"class": "bp3-card player"})
            select2 = select.find_all("div", {"class": "column col-3"})
            driver.close()
            return_ = int(select2[0].get_text().split(" ")[0])
    except:
        pass
    
    return return_

def modify_apostrophe(elem):
    
    elem = elem.replace("'","").lower()
    elem = elem.split(" ")
    elem[0] = elem[0].capitalize()
    elem[1] = elem[1].capitalize()
    elem = elem[0] + " " + elem[1]
    return elem

def match_link(link, pitch, compo):

    tab = []
    for elem in compo["lineup"][pitch]:
        """apos = bool(re.search(r"'", elem))
        if apos:
            elem = modify_apostrophe(elem)"""
        elem = elem.replace("'", "")
        elem = elem.split(" ")
        if len(elem) == 2:
            
            try: 
                tab.append(link[f" {unidecode.unidecode(elem[0][0])}. {elem[1]}"])
            except:
                try:
                    tab.append(link[f" {elem[1]}"])
                except:
                    try:
                        tab.append(link[f" {elem[0]}"])
                    except:
                        try:
                            tab.append(link[f" {unidecode.unidecode(elem[0][0])}. {unidecode.unidecode(elem[1])}"])     
                        except:
                            try:
                                tab.append(link[f" {unidecode.unidecode(elem[0][0])}. {elem[1].lower().capitalize()}"])  
                            except:
                                try:
                                    tab.append(link[f" {unidecode.unidecode(elem[0])} {unidecode.unidecode(elem[1].lower().capitalize())}"]) 
                                except:
                                    try:
                                        tab.append(link[f" {elem[0].split('-')[1][0]}. {unidecode.unidecode(elem[1])}"])
                                    except:
                                        try:
                                            split = elem[1].split("-")
                                            tab.append(link[f" {unidecode.unidecode(elem[0][0])}. {unidecode.unidecode(split[0])} {unidecode.unidecode(split[1])}"]) 
                                        except:
                                            try:
                                                tab.append(link[f" {elem[0]} {elem[1]}"])
                                            except:
                                                pass

        elif len(elem) == 3:
            
            try:
                tab.append(link[f" {unidecode.unidecode(elem[0][0])}. {unidecode.unidecode(elem[1])} {unidecode.unidecode(elem[2])}"])
            except:
                try:
                    tab.append(link[f" {unidecode.unidecode(elem[0][0])}. {unidecode.unidecode(elem[2])}"])
                except:
                    try:
                        tab.append(link[f" {unidecode.unidecode(elem[1][0])}. {unidecode.unidecode(elem[2])}"])
                    except:
                        try:
                            tab.append(link[f" {unidecode.unidecode(elem[0])} {unidecode.unidecode(elem[1])} {unidecode.unidecode(elem[2])}"])
                        except:
                            try:
                                #print(f" {unidecode.unidecode(elem[0])} {unidecode.unidecode(elem[1].lower().capitalize())} {unidecode.unidecode(elem[2])}")
                                tab.append(link[f" {unidecode.unidecode(elem[0][0])}. {unidecode.unidecode(elem[1].lower().capitalize())} {unidecode.unidecode(elem[2])}"])
                            except:
                                pass
        elif len(elem) == 4:
            try:
                tab.append(link[f" {unidecode.unidecode(elem[0][0])}. {unidecode.unidecode(elem[2])}"])
            except:
                pass

        else:
            try:  
                tab.append(link[f" {elem[0]}"])
            except:
                try:
                    tab.append(link[f" {elem[0]} Jr"])
                except:
                    try:
                        if elem[0] == "Memphis":
                            tab.append(link[" M. Depay"])
                        elif elem[0] == "Boschilia":
                            tab.append(link[" Gabriel Boschilia"])        
                    except:
                        pass
                             
    return tab

def get_team(soup):
    
    tab = []
    select = soup.find_all("a", {"class": "team-link"})
    for elem in select:
        tab.append(elem.get_text())
    return tab

def get_manager(soup):
    
    tab = []
    select = soup.find_all("span", {"class": "manager-name"})
    for elem in select:
        tab.append(elem.get_text())
    return tab

def format_data(col_names, compo):
    
    tab = []
    for key in compo.keys():
        if key in ["lineup", "OR"]:
            for i in [0,1]:
                for elem in compo[key][i]:
                    tab.append(elem)
        elif key in ["team", "manager","formation", "score"]:
            for elem in compo[key]:
                   tab.append(elem)
        else:
            tab.append(compo[key])
    return {key:value for key, value in zip(col_names, tab)}

def get_row(html):
    
    compo = parse_file(html, 1)
    compo["OR"] = [[], []]
   
    homelink = get_link_player(team[compo["team"][0]], compo["date_id"], 0)
    homelink = match_link(homelink,0, compo)
    if len(homelink) != 11:
        homelink = get_link_player(team[compo["team"][0]], compo["date_id"], 1)
        homelink = match_link(homelink,0, compo)
    for elem in homelink:
        elem = elem.split("/")
        link = f"https://sofifa.com/player/{elem[2]}/{elem[3]}/?r={elem[4]}&set=true"
        compo["OR"][0].append(get_stat(link))
        
   
    awaylink = get_link_player(team[compo["team"][1]], compo["date_id"], 0)
    awaylink = match_link(awaylink,1, compo)
    if len(awaylink) != 11:
        awaylink = get_link_player(team[compo["team"][1]], compo["date_id"], 1)
        awaylink = match_link(awaylink,1, compo)
    for elem in awaylink:
        elem = elem.split("/")
        link = f"https://sofifa.com/player/{elem[2]}/{elem[3]}/?r={elem[4]}&set=true"
        compo["OR"][1].append(get_stat(link))
        
    return compo

In [24]:
links = ["https://fr.whoscored.com/Matches/1281213/Live/France-Ligue-1-2018-2019-Marseille-Toulouse",
         "https://fr.whoscored.com/Matches/1281215/Live/France-Ligue-1-2018-2019-Nantes-Monaco",
         "https://fr.whoscored.com/Matches/1281216/Live/France-Ligue-1-2018-2019-Nice-Reims",
         "https://fr.whoscored.com/Matches/1281214/Live/France-Ligue-1-2018-2019-Montpellier-Dijon",
         "https://fr.whoscored.com/Matches/1281209/Live/France-Ligue-1-2018-2019-Angers-Nimes",
         "https://fr.whoscored.com/Matches/1281211/Live/France-Ligue-1-2018-2019-Lille-Rennes",
         "https://fr.whoscored.com/Matches/1281218/Live/France-Ligue-1-2018-2019-Saint-Etienne-Guingamp",
         "https://fr.whoscored.com/Matches/1281212/Live/France-Ligue-1-2018-2019-Lyon-Amiens",
         "https://fr.whoscored.com/Matches/1281210/Live/France-Ligue-1-2018-2019-Bordeaux-Strasbourg",
         "https://fr.whoscored.com/Matches/1281217/Live/France-Ligue-1-2018-2019-Paris-Saint-Germain-Caen",
         "https://fr.whoscored.com/Matches/1281236/Live/France-Ligue-1-2018-2019-Reims-Lyon",
         "https://fr.whoscored.com/Matches/1281230/Live/France-Ligue-1-2018-2019-Guingamp-Paris-Saint-Germain",
         "https://fr.whoscored.com/Matches/1281232/Live/France-Ligue-1-2018-2019-Monaco-Lille",
         "https://fr.whoscored.com/Matches/1281238/Live/France-Ligue-1-2018-2019-Rennes-Angers",
         "https://fr.whoscored.com/Matches/1281219/Live/France-Ligue-1-2018-2019-Amiens-Montpellier",
         "https://fr.whoscored.com/Matches/1281220/Live/France-Ligue-1-2018-2019-Caen-Nice",
         "https://fr.whoscored.com/Matches/1281221/Live/France-Ligue-1-2018-2019-Dijon-Nantes",
         "https://fr.whoscored.com/Matches/1281240/Live/France-Ligue-1-2018-2019-Strasbourg-Saint-Etienne",
         "https://fr.whoscored.com/Matches/1281222/Live/France-Ligue-1-2018-2019-Toulouse-Bordeaux",
         "https://fr.whoscored.com/Matches/1281234/Live/France-Ligue-1-2018-2019-Nimes-Marseille",
         "https://fr.whoscored.com/Matches/1281226/Live/France-Ligue-1-2018-2019-Lyon-Strasbourg",
         "https://fr.whoscored.com/Matches/1281233/Live/France-Ligue-1-2018-2019-Paris-Saint-Germain-Angers",
         "https://fr.whoscored.com/Matches/1281228/Live/France-Ligue-1-2018-2019-Montpellier-Saint-Etienne",
         "https://fr.whoscored.com/Matches/1281229/Live/France-Ligue-1-2018-2019-Nantes-Caen",
         "https://fr.whoscored.com/Matches/1281231/Live/France-Ligue-1-2018-2019-Nice-Dijon",
         "https://fr.whoscored.com/Matches/1281223/Live/France-Ligue-1-2018-2019-Amiens-Reims",
         "https://fr.whoscored.com/Matches/1281235/Live/France-Ligue-1-2018-2019-Toulouse-Nimes",
         "https://fr.whoscored.com/Matches/1281225/Live/France-Ligue-1-2018-2019-Lille-Guingamp",
         "https://fr.whoscored.com/Matches/1281224/Live/France-Ligue-1-2018-2019-Bordeaux-Monaco",
         "https://fr.whoscored.com/Matches/1281227/Live/France-Ligue-1-2018-2019-Marseille-Rennes"]

error = []
tab = []
for link in links:
    driver  = webdriver.Chrome(PATH)
    driver.get(link)
    delay = 5
    try:
        myElem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.ID, "match-header")))
        print("Page is ready!")
        html = driver.page_source
    except TimeoutException:
        print("Loading took too much time")
    driver.close()
    try:
        tab.append(format_data(col_names, get_row(html)))
    except Exception:
        traceback.print_exc()
        error.append(link)
tab = pd.DataFrame(tab)
tab.to_csv("test.csv")

tab

Page is ready!


,homeTeam,awayTeam,homeManager,awayManager,homeGK,homeP1,homeP2,homeP3,homeP4,homeP5,homeP6,homeP7,homeP8,homeP9,homeP10,awayGK,awayP1,awayP2,awayP3,awayP4,awayP5,awayP6,awayP7,awayP8,awayP9,awayP10,homeFormation,awayFormation,date,dateId,homeScore,awayScore,ORHomeGK,ORHomeP1,ORHomeP2,ORHomeP3,ORHomeP4,ORHomeP5,ORHomeP6,ORHomeP7,ORHomeP8,ORHomeP9,ORHomeP10,ORAwayGK,ORAwayP1,ORAwayP2,ORAwayP3,ORAwayP4,ORAwayP5,ORawayP6,ORAwayP7,ORAwayP8,ORAwayP9,ORAwayP10
0,Lyon,Amiens,Bruno Génésio,Christophe Pelissier,Mathieu Gorgelin,Léo Dubois,Marcelo,Jérémy Morel,Ferland Mendy,Tanguy Ndombele,Lucas Tousart,Houssem Aouar,Memphis,Bertrand Traoré,Mariano,Régis Gurtner,Oualid El Hajjam,Khaled Adénon,Bakaye Dibassy,Jordan Lefort,Guessouma Fofana,Eddy Gnahore,Gaoussou Traore,Juan Ferney Otero,Moussa Konaté,John Steven Mendoza,4-3-1-2,4-3-3,"Aug 12, 2018",190001,2,0,68,77,81,77,77,78,79,78,83,79,80,77,72,73,73,66,71,69,62,73,76,70
